In [3]:
import pickle

with open('aligned_paragraph_dataset.pickle', 'rb') as fp:
  aligned_paragraph_dataset = pickle.load(fp)

with open('source_paragraph_dataset.pickle', 'rb') as fp:
  source_paragraph_dataset = pickle.load(fp)

In [4]:
from nltk.tokenize import sent_tokenize
import pandas as pd
from thefuzz import fuzz
import random

## English Idioms in Translations

In [5]:
en_df = pd.read_json("/home/kkatsy/idioms/magpie-corpus/MAGPIE_unfiltered.jsonl", lines=True)
en_df.head()

,confidence,context,document_id,genre,id,idiom,judgment_count,label,label_distribution,non_standard_usage_explanations,offsets,sentence_no,split,variant_type
0,1.0,"[, , One can not come to terms with the past ,...",p39d1118,PMB,0,come to terms with,3,i,"{'?': 0.0, 'f': 0.0, 'i': 1.0, 'l': 0.0, 'o': ...",[],"[[12, 16], [17, 19], [20, 25], [26, 30]]",0,training,identical
1,1.0,[And there may be one or two other things we w...,J97,S meeting,1,call it a day,3,i,"{'?': 0.0, 'f': 0.0, 'i': 1.0, 'l': 0.0, 'o': ...",[],"[[9, 13], [14, 16], [19, 22]]",470,training,identical
2,1.0,"[It was the first thing I asked.’, Lindsey nod...",JXW,W fict prose,2,out of the blue,3,i,"{'?': 0.0, 'f': 0.0, 'i': 1.0, 'l': 0.0, 'o': ...",[],"[[77, 80], [81, 83], [88, 92]]",1202,training,identical
3,1.0,[When I took these two guitars into the studio...,C9M,W pop lore,3,full tilt,5,i,"{'?': 0.0, 'f': 0.0, 'i': 1.0, 'l': 0.0, 'o': ...",[],"[[234, 238], [239, 243]]",1117,training,identical
4,1.0,"[Now , generally speaking wi I I I 've act wha...",JT1,S lect soc science,4,on paper,3,l,"{'?': 0.0, 'f': 0.0, 'i': 0.0, 'l': 1.0, 'o': ...",[],"[[102, 104], [115, 120]]",291,training,insertion-other


In [24]:
en_idioms = list(en_df["idiom"].astype('str'))
print(en_idioms[:10])

print('Num idioms: ', len(en_idioms))

['come to terms with', 'call it a day', 'out of the blue', 'full tilt', 'on paper', 'to the letter', 'strike a chord', 'muddy the waters', 'do the honours', 'high and low']
Num idioms:  56622


In [7]:
! pip install thefuzz

In [8]:
name = "Kurtis Pykes"
full_name = "hello my name is Kurtis K D Pykes and i like potatoes"

print(f"Token sort ratio similarity score: {fuzz.partial_ratio(name, full_name)}")

Token sort ratio similarity score: 67


In [32]:
count = 0

for id in random.sample(en_idioms, 500):
    for book in sorted(list(source_paragraph_dataset.keys())):
        num_pars = len(source_paragraph_dataset[book])
        for i in range(num_pars):
            # id needs to be less than par
            translators = aligned_paragraph_dataset[book][i].keys()
            for t in translators:
                par = aligned_paragraph_dataset[book][i][t]
                sents = sent_tokenize(par)
                for s in sents:
                    if len(s) > 10 and len(id.split(' ')) > 0 and len(id) < len(s):
                        fuzzy = fuzz.partial_ratio(id, s)
                        if fuzzy > 95:
                            count +=1
                            print('Num ', count)
                            print('score: ', fuzzy)
                            print('idiom: ', id)
                            print('sent: ', s)
                            print('book: ', book, ', par idx: ', i)
                            print('translator: ', t, '\n')
    if count > 50:
        break

Num  1
score:  100
idiom:  under lock and key
sent:  I'm going to pick you up, put you under my arm, take you home and place you under lock and key!'
book:  CrimeAndPunishment , par idx:  1092
translator:  McDuff 

Num  2
score:  100
idiom:  under lock and key
sent:  Arkady Ivanovich got up, laughed, kissed his fiancée, patted her on the cheek, assured her he would be back soon and, noticing in her eyes not only a certain childish curiosity but also a very earnest, unspoken question, thought for a moment, gave her another kiss and as he did so experienced a pang of sincere regret within his soul that the gift would immediately be placed under lock and key for safekeeping by that wisest of mothers.
book:  CrimeAndPunishment , par idx:  3573
translator:  McDuff 

Num  3
score:  100
idiom:  under lock and key
sent:  The main thing was, however, that they should not tell anyone about all this, because heaven only knew what might happen then, and the money must be immediately placed under l

### Examples:

In [36]:
idx = 1274
book = 'TheIdiot'
idiom = 'on tenterhooks'
translator = 'Garnett'

target_keys = aligned_paragraph_dataset[book][idx]
source = source_paragraph_dataset[book][idx]

print('Source:')
print(source)
print('Translations:')
for t in target_keys:
    print(t, ': ', aligned_paragraph_dataset[book][idx][t])


Source:
Все устремили взгляды на Птицына, читавшего письмо. Общее любопытство получило новый и чрезвычайный толчок. Фердыщенку не сиделось; Рогожин смотрел в недоумении и в ужасном беспокойстве переводил взгляды то на князя, то на Птицына. Дарья Алексеевна в ожидании была как на иголках. Даже Лебедев не утерпел, вышел из своего угла, и, согнувшись в три погибели, стал заглядывать в письмо чрез плечо Птицына, с видом человека, опасающегося, что ему сейчас дадут за это колотушку.
Translations:
Garnett :  Everyone fixed their eyes on Paityn as he read the letter. The general curiosity had received a new and violent stimulus. Ferdyshtchenko could not keep still; Rogozhin looked on with amazement and great anxiety, turning his eyes from Myshkin to Paityn. Darya Alexeyevna seemed on tenterhooks of expectation. Even Lebedyev could not help coming out of his corner and bending himself into a triangle, peeped at the letter over Ptitsyn's shoulder with the air of aman expecting a  blow for doing

In [40]:
# idx = 3673
# book = 'Demons'
# idiom = 'under lock and key'
# translator = ''

# print('Source:')
# print(source)
# print('Translations:')
# for t in target_keys:
#     print(t, ': ', aligned_paragraph_dataset[book][idx][t])

In [41]:
# idx = 4956
# book = 'TheBrothersKaramazov'
# idiom = 'in light of'
# translator = ''

# print('Source:')
# print(source)
# print('Translations:')
# for t in target_keys:
#     print(t, ': ', aligned_paragraph_dataset[book][idx][t])

## Russian Idioms in Source Texts

In [25]:
with open('/home/kkatsy/idioms/ru_idioms.txt', "r") as fp:
    ru_id_lines = fp.readlines()

ru_idioms = [line.strip().lower() for line in ru_id_lines]

print(ru_idioms[:10])

print('Num idioms: ', len(ru_idioms))

['авось', 'ахиллесова пята', 'бабушка ещё надвое гадала', 'базарная баба', 'без всякой задней мысли', 'без году неделю', 'без задней мысли', 'без задних ног', 'без мыла влезать в душу', 'без оглядки']
Num idioms:  1651


In [20]:
count = 0
import random

for id in random.sample(ru_idioms, 500):
    for book in sorted(list(source_paragraph_dataset.keys())):
        num_pars = len(source_paragraph_dataset[book])
        for i in range(num_pars):
            # id needs to be less than par
            par = source_paragraph_dataset[book][i]
            sents = sent_tokenize(par)
            for s in sents:
                if len(s) > 10 and len(id.split(' ')) > 0 and len(id) < len(s):
                    fuzzy = fuzz.partial_ratio(id, s)
                    if fuzzy > 90:
                        count +=1
                        print('Num ', count)
                        print('score: ', fuzzy)
                        print('idiom: ', id)
                        print('sent: ', s)
                        print('book: ', book, ', par idx: ', i, '\n')
    if count > 50:
        break
            

Num  1
score:  100
idiom:  испокон века
sent:  У него нет достаточно земли, – ну, он и захватил чужую пустошь, то есть он рассчитывал, что она не нужна, и о ней хозяева <забыли>, а у нас, как нарочно, уже испокон века собираются крестьяне праздновать там Красную горку.
book:  DeadSouls , par idx:  1899 

Num  2
score:  92
idiom:  испокон века
sent:  Наконец, и главное, конечно для того, чтоб его, Смердякова, разбитого припадком, тотчас же перенесли из кухни, где он всегда отдельно ото всех ночевал и где имел свой особенный вход и выход, в другой конец флигеля, в комнатку Григория, к ним обоим за перегородку, в трех шагах от их собственной постели, как всегда это бывало, спокон века, чуть только его разбивала падучая, по распоряжениям барина и сердобольной Марфы Игнатьевны.
book:  TheBrothersKaramazov , par idx:  5105 

Num  3
score:  100
idiom:  на ногах
sent:  Николай Щербацкий, двоюродный брат Кити, в коротенькой жакетке и узких панталонах, сидел с коньками на ногах на скамейке и, ув

### Examples:

In [42]:
idx = 4133
book = 'TheIdiot'
idiom = 'вне себя'

target_keys = aligned_paragraph_dataset[book][idx]
source = source_paragraph_dataset[book][idx]

print('Source:')
print(source)
print('Translations:')
for t in target_keys:
    print(t, ': ', aligned_paragraph_dataset[book][idx][t])

Source:
Князь был решительно вне себя.
Translations:
Garnett :  Myshkin had completely lost control of himself.
McDuff :  The prince was well and truly beside himself.
PV :  The prince was decidedly beside himself.


In [43]:
idx = 3405
book = 'TheIdiot'
idiom = 'как снег на голову'

target_keys = aligned_paragraph_dataset[book][idx]
source = source_paragraph_dataset[book][idx]

print('Source:')
print(source)
print('Translations:')
for t in target_keys:
    print(t, ': ', aligned_paragraph_dataset[book][idx][t])

Source:
– Н-нет-с; денег я не давал-с, и он сам знает, что я и не дам-с, но ведь единственно в видах воздержания и исправления его. Теперь увязался со мной в Петербург; я в Петербург ведь еду-с, чтобы застать господина Фердыщенка по самым горячим следам, ибо наверно знаю, что он уже там-с. Генерал мой так и кипит-с; но подозреваю, что в Петербурге улизнет от меня, чтобы посетить капитаншу. Я, признаюсь, даже нарочно его от себя отпущу, как мы уже и условились по приезде тотчас же разойтись в разные стороны, чтоб удобнее изловить господина Фердыщенка. Так вот я его отпущу, а потом вдруг, как снег на голову, и застану его у капитанши, – собственно, чтоб его пристыдить, как семейного человека и как человека вообще говоря.
Translations:
Garnett :  "N-no; money I have not given him, and he knows himself that I won't give it him. But that's solely with a view to his elevation and reformation. Now he is insisting on coming to Petersburg with me. You see, I'm going to Petersburg to find Mr. Fe

In [45]:
idx = 571
book = 'CrimeAndPunishment'
idiom = 'на ногах'

target_keys = aligned_paragraph_dataset[book][idx]
source = source_paragraph_dataset[book][idx]

print('Source:')
print(source)
print('Translations:')
for t in target_keys:
    print(t, ': ', aligned_paragraph_dataset[book][idx][t])

Source:
– Он едва на ногах стоит, а ты… – заметил было Никодим Фомич.
Translations:
Katz :  "He can hardly stand, while you think. . ." remarked Nikodim Fomich.
Garnett :  "He can scarcely stand upright. And you…" Nikodim Fomitch was beginning.
McDuff :  'He can hardly stay standing, yet you ...' Nikodim Fomich began to protest.
PV :  "He can barely stand on his feet, and you..." Nikodim Fomich began.


## Dostoevsky's Literary Idioms

In [28]:
with open('/home/kkatsy/idioms/dostoevsky_idioms.txt', "r") as fp:
    ru_id_lines = fp.readlines()

dost_idioms = [line.strip().lower() for line in ru_id_lines]

print(dost_idioms[:5])

print('Num idioms: ', len(dost_idioms))

['бабушкин сoн', 'бедность не порок', 'без гроша', 'без задних ног', 'без зазрения совести']
Num idioms:  102


In [16]:
count = 0
for id in dost_idioms:
    for book in sorted(list(source_paragraph_dataset.keys())):
        num_pars = len(source_paragraph_dataset[book])
        for i in range(num_pars):
            # id needs to be less than par
            par = source_paragraph_dataset[book][i]
            sents = sent_tokenize(par)
            for s in sents:
                if len(s) > 10 and len(id.split(' ')) > 0 and len(id) < len(s):
                    fuzzy = fuzz.partial_ratio(id, s.lower())
                    if fuzzy > 90:
                        count +=1
                        print('Num ', count)
                        print('score: ', fuzzy)
                        print('idiom: ', id)
                        print('sent: ', s)
                        print('book: ', book, ', par idx: ', i, '\n')

Num  1
score:  92
idiom:  бабушкин сoн
sent:  Ну, конечно, бабушкин сон рассказывает, врет, как лошадь, потому я этого Душкина знаю, сам он закладчик и краденое прячет, и тридцатирублевую вещь не для того, чтоб «преставить», у Миколая подтибрил.
book:  CrimeAndPunishment , par idx:  815 

Num  2
score:  100
idiom:  бедность не порок
sent:  – Милостивый государь, – начал он почти с торжественностию, – бедность не порок, это истина.
book:  CrimeAndPunishment , par idx:  60 

Num  3
score:  100
idiom:  бедность не порок
sent:  – Бедность не порок, дружище, ну да уж что!
book:  CrimeAndPunishment , par idx:  528 

Num  4
score:  100
idiom:  бедность не порок
sent:  Будьте благородным человеком, твердым в несчастиях; помните, что бедность не порок.
book:  PoorFolk , par idx:  270 

Num  5
score:  100
idiom:  без гроша
sent:  Прислала бы и больше, но боюсь за наши расходы дорожные; хотя Петр Петрович был так добр, что взял на себя часть издержек по нашему проезду в столицу, а именно сам вызв

### Examples:

In [46]:
idx = 2519
book = 'CrimeAndPunishment'
idiom = 'время терпят'

target_keys = aligned_paragraph_dataset[book][idx]
source = source_paragraph_dataset[book][idx]

print('Source:')
print(source)
print('Translations:')
for t in target_keys:
    print(t, ': ', aligned_paragraph_dataset[book][idx][t])

Source:
– Да-да-да! Не беспокойтесь! Время терпит, время терпит-с, – бормотал Порфирий Петрович, похаживая взад и вперед около стола, но как-то без всякой цели, как бы кидаясь то к окну, то к бюро, то опять к столу, то избегая подозрительного взгляда Раскольникова, то вдруг сам останавливаясь на месте и глядя на него прямо в упор. Чрезвычайно странною казалась при этом его маленькая, толстенькая и круглая фигурка, как будто мячик, катавшийся в разные стороны и тотчас отскакивавший от всех стен и углов.
Translations:
Katz :  "Yes, yes, yes! Don't worry! There's no hurry, no hurry at all, sir," muttered Porfiry Petrovich, pacing back and forth around the table, but seemingly without any objective, as if hastening first to the window, then to the bureau, then back to the table, first avoiding Raskolnikov's suspicious glance, then suddenly stopping in one place and staring directly at him. His small, plump, round figure, like a ball rolling in various directions and bouncing off all the wa

In [48]:
idx = 2936
book = 'TheIdiot'
idiom = 'во что бы то ни стало'

target_keys = aligned_paragraph_dataset[book][idx]
source = source_paragraph_dataset[book][idx]

print('Source:')
print(source)
print('Translations:')
for t in target_keys:
    print(t, ': ', aligned_paragraph_dataset[book][idx][t])

Source:
«Идея о том (продолжал он читать), что не стоит жить несколько недель, стала одолевать меня настоящим образом, я думаю, с месяц назад, когда мне оставалось жить еще четыре недели, но совершенно овладела мной только три дня назад, когда я возвратился с того вечера в Павловске. Первый момент полного, непосредственного проникновения этою мыслью произошел на террасе у князя, именно в то самое мгновение, когда я вздумал сделать последнюю пробу жизни, хотел видеть людей и деревья (пусть это я сам говорил), горячился, настаивал на праве Бурдовского, „моего ближнего“, и мечтал, что все они вдруг растопырят руки и примут меня в свои объятия, и попросят у меня в чем-то прощения, а я у них; одним словом, я кончил как бездарный дурак. И вот в эти-то часы и вспыхнуло во мне „последнее убеждение“. Удивляюсь теперь, каким образом я мог жить целые шесть месяцев без этого „убеждения“! Я положительно знал, что у меня чахотка, и неизлечимая; я не обманывал себя и понимал дело ясно. Но чем яснее я

In [49]:
idx = 1040
book = 'Demons'
idiom = 'во все глаза'

target_keys = aligned_paragraph_dataset[book][idx]
source = source_paragraph_dataset[book][idx]

print('Source:')
print(source)
print('Translations:')
for t in target_keys:
    print(t, ': ', aligned_paragraph_dataset[book][idx][t])

Source:
– Я… я думала, так надо, – пролепетала она, смотря во все глаза на Варвару Петровну, – так вас Лиза звала.
Translations:
Garnett :  "I... I... thought that was the proper way," she faltered, gazing open-eyed at Varvara Petrovna. "Liza called you that."
PV :  "I ... [thought that's how it should be," she prattled, staring at Varvara Petrovna, "that's what Liza called you."
